In [1]:
from collections import defaultdict
from pprint import pprint

import gensim
import numpy as np
import pandas as pd
from surprise import Dataset, Reader
from tqdm import tqdm

from src.models import cf

tqdm.pandas()

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# Load Data

In [2]:
# global variables
DATA_PATH = "data/evaluation"
CATEGORY = "Pet_Supplies"

# training parameters
N_EPOCHS = 10
LR_ALL = 0.005
BETA = 0.1

train = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_train.csv")

In [3]:
# checking train dataframe
train.head().append(train.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,0,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A14CK12J7C7JRK,3.0,I purchased the Trilogy with hoping my two cat...,2011-01-12,purchase trilogy hop cat age interested yr old...
1,2,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2CR37UY3VR7BN,4.0,I bought the triliogy and have tested out all ...,2012-12-19,buy triliogy test dvd appear volume receive re...
2,3,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2A4COGL9VW2HY,4.0,My female kitty could care less about these vi...,2011-05-12,female kitty care video care little male dig a...
3,4,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A2UBQA85NIGLHA,3.0,"If I had gotten just volume two, I would have ...",2012-03-05,volume star trilogy star read review know vol ...
4,5,B00005MF9U,LitterMaid LM900 Mega Self-Cleaning Litter Box,"['Pet Supplies', 'Cats', 'Litter &amp; Housebr...",A2BH04B9G9LOYA,1.0,"First off, it seems that someone is spamming t...",2006-12-31,spamming review glow reviewer review amazon ba...
68865,111581,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A11J1FHCK5U06J,4.0,Now I know exactly where the trouble spots are...,2014-05-23,know exactly trouble spot sniffing guess invis...
68866,111585,B00K3YPOO0,Brightest Black Light Flashlight on Amazon- UV...,[],A18JF0T0GOCORW,4.0,I use this light to help me find stains when I...,2014-05-24,use light help stain carpet clean pre treat ca...
68867,111595,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A3GRPCW9DG427Z,5.0,We are owned by the 3 pickiest pooches in the ...,2013-07-27,pickiest pooch world love fool reject doggie t...
68868,111598,B00K7EG97C,Nutro Crunchy Dog Treats with Real Mixed Berri...,"['Pet Supplies', 'Dogs', 'Treats', 'Cookies, B...",A2X6TLAX3JEO1A,5.0,My highly allergic white boxer loves these tre...,2014-05-09,highly allergic white boxer love treat meat co...
68869,111602,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A9PG9ODPPP31N,5.0,Works great on my medium sized dog. She has ve...,2014-07-09,work great medium size dog coarse hair work gr...


# Preparing Topic Vectors

In [4]:
class LDA:
    def __init__(self, reviews):
        self.reviews = reviews
        self.lda = None
        self.dictionary = None

    def train(self, n_topics=50, n_epochs=20, workers=8):
        # tokenizations
        dictionary = gensim.corpora.Dictionary(self.reviews)
        # filtering tokens less than 5 reviews, more than 0.85 reviews
        dictionary.filter_extremes(no_below=5, no_above=0.85)
        # creating dict how many words and time it appears
        bow_corpus = [dictionary.doc2bow(doc) for doc in self.reviews]
        
        # train model
        self.lda = gensim.models.LdaMulticore(bow_corpus, 
                                              num_topics=n_topics, 
                                              id2word=dictionary, 
                                              passes=n_epochs,
                                              workers=workers)
        # save dictionary
        self.dictionary = dictionary
        
    def get_document_topics(self, doc, minimum_probability=0.0):
        """
        """
        return self.lda.get_document_topics(doc, minimum_probability=minimum_probability)

In [5]:
# generating tokenized reviews
processed_reviews = train["processedReviewText"].progress_apply(lambda x: x.split())

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 68870/68870 [00:00<00:00, 181937.33it/s]


In [6]:
# instantiate lda model
lda_model = LDA(processed_reviews)

In [7]:
%%time
# training the LDA model
lda_model.train()

/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/jensen/Desktop/recommendation-systems/projects/product-reviews-recommender/.venv/lib/python3.9/site-packages/gensim/similar

CPU times: user 2min 39s, sys: 28.3 s, total: 3min 7s
Wall time: 3min 52s


# Generating User/Item Topic Vectors

In [8]:
def get_topic_vectors(model, corpus, n_topics=50):
    """
    """
    topic_vecs = []
    for i in tqdm(range(len(corpus))):
        top_topics = model.get_document_topics(corpus[i])
        topic_vecs.append([top_topics[i][1] for i in range(n_topics)])
        
    return topic_vecs

def generate_user_item_vectors(lda: LDA, train: pd.DataFrame):
    """
    """
    user_reviews = train.groupby(["reviewerID"])['processedReviewText'].apply(lambda x: ' '.join(x))
    item_reviews = train.groupby(["asin"])["processedReviewText"].apply(lambda x: ' '.join(x))
    
    # get unique users and items
    unique_users = user_reviews.index.tolist()
    unique_items = item_reviews.index.tolist()
    
    # tokenize reviews
    user_reviews_list = user_reviews.apply(lambda x: x.split()).tolist()
    item_reviews_list = item_reviews.apply(lambda x: x.split()).tolist()
    
    # generate corpus based on aggregate of user/item reviews
    user_corpus = [lda.dictionary.doc2bow(doc) for doc in user_reviews_list]
    item_corpus = [lda.dictionary.doc2bow(doc) for doc in item_reviews_list]
    
    # retrieve user and item topics vectors
    user_vecs = get_topic_vectors(lda, user_corpus)
    item_vecs = get_topic_vectors(lda, item_corpus)
    
    # generate a mapping 
    user_idx_map = {k: unique_users[k] for k in range(len(unique_users))}
    item_idx_map = {k: unique_items[k] for k in range(len(unique_items))}
    user_vec_map = {k: v for k, v in zip(unique_users, user_vecs)}
    item_vec_map = {k: v for k, v in zip(unique_items, item_vecs)}
    
    # loading user topic vectors into DF
    user_vecs = pd.DataFrame.from_dict(user_vec_map, orient='index')
    user_vecs.index.name = 'reviewerID'
    # loading item topic vectors into DF
    item_vecs = pd.DataFrame.from_dict(item_vec_map, orient='index')
    item_vecs.index.name = 'asin'
    
    return user_idx_map, user_vecs, item_idx_map, item_vecs

In [9]:
user_idx_map, user_vecs, item_idx_map, item_vecs = generate_user_item_vectors(lda_model, train)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4878/4878 [00:06<00:00, 756.40it/s]


In [10]:
# converting factors into numpy obj
user_factors = user_vecs.to_numpy()
item_factors = item_vecs.to_numpy()

In [11]:
# check user factors
user_factors[0,:]

array([0.00036378, 0.13002966, 0.00036378, 0.00036378, 0.00036378,
       0.19430454, 0.00036378, 0.00036378, 0.00036378, 0.00036378,
       0.00036378, 0.00036378, 0.00036378, 0.00036378, 0.00036378,
       0.00036378, 0.00036378, 0.00036378, 0.00036378, 0.00036378,
       0.00036378, 0.00036378, 0.07538441, 0.00036378, 0.11798983,
       0.00036378, 0.00036378, 0.00036378, 0.00036378, 0.00036378,
       0.00036378, 0.00036378, 0.00036378, 0.00036378, 0.00036378,
       0.00036378, 0.00036378, 0.05412404, 0.20626469, 0.00036378,
       0.12013067, 0.00036378, 0.00036378, 0.08649324, 0.00036378,
       0.00036378, 0.00036378, 0.00036378, 0.00036378, 0.00036378],
      dtype=float32)

In [12]:
# check item factors
item_factors[0,:]

array([2.6423439e-01, 1.4528287e-04, 2.6851520e-02, 4.5869824e-02,
       1.4528287e-04, 1.4528287e-04, 1.4528287e-04, 1.6984434e-01,
       1.4528287e-04, 1.4528287e-04, 1.4528287e-04, 1.4528287e-04,
       1.4528287e-04, 1.4528287e-04, 1.4528287e-04, 1.4528287e-04,
       1.4528287e-04, 1.4528287e-04, 1.4528287e-04, 1.4528287e-04,
       3.3183634e-02, 1.4528287e-04, 2.3089450e-02, 1.4528287e-04,
       1.4528287e-04, 1.4528287e-04, 1.4528287e-04, 1.4528287e-04,
       2.8263895e-02, 1.4557328e-02, 1.1008586e-01, 1.4528287e-04,
       1.4528287e-04, 6.3512892e-02, 9.6558370e-02, 1.4528287e-04,
       1.4528287e-04, 1.2390847e-02, 3.9634366e-02, 1.4528287e-04,
       1.4528287e-04, 1.4528287e-04, 1.4528287e-04, 1.4528287e-04,
       1.4528287e-04, 6.6693068e-02, 1.4528287e-04, 1.4528287e-04,
       1.4528287e-04, 1.4528287e-04], dtype=float32)

# Utility Functions

In [13]:
def get_top_n(predictions, n=10):
    """Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    """

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in tqdm(predictions):
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in tqdm(top_n.items()):
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

def recall_at_k(asins, predicted_asins, k=10):
    # number of relevant items
    set_actual = set(asins)
    set_preds = set(predicted_asins)
    num_relevant = len(set_actual.intersection(set_preds))
    
    # calculating recall@K - relevant / total relevant items
    recall_at_k = num_relevant / len(asins)
    
    return recall_at_k

def novelty_at_k(item_popularity, predicted_asins, k=10):
    """
    """
    # finding avg novelty
    popularity_sum = item_popularity.loc[predicted_asins].sum()
    novelty_at_k = ((k*1) - popularity_sum) / k
    
    return novelty_at_k

def generate_item_popularity(train: pd.DataFrame) -> pd.DataFrame:
    """
    """
    
    # create a mapping of item popularatity
    # based on sum(item's review / max reviews) / no items
    max_reviews = (train.groupby(['asin'])
                   .agg({'processedReviewText': 'count'})
                   .max()
                   .values[0])
    item_popularity = (train.groupby(['asin'])
                       .agg({'processedReviewText': 'count'})
                       .apply(lambda x: x/max_reviews))
    
    return item_popularity
    

def evaluate_recommendations(top_ns: dict, user_rating_history: pd.DataFrame, item_popularity: pd.DataFrame, k=10) -> pd.DataFrame:
    """
    
    Args:
        top_ns
        user_rating_history
    """
    
    test_recommendations = pd.DataFrame(top_ns.items(), columns=["reviewerID", "pred_asin"])
    test_recommendations['pred_asin'] = test_recommendations['pred_asin'].apply(lambda x: [i[0] for i in x])
    
    # combined test history and recommendations
    test_merged = pd.merge(user_rating_history, test_recommendations, on="reviewerID", how="inner")
    
    # generating recall@k metrics
    test_merged["recall@k"] = test_merged.apply(lambda x: recall_at_k(x.asin, x.pred_asin, k=k), axis=1)
    test_merged["novelty@k"] = test_merged.apply(lambda x: novelty_at_k(item_popularity, x.pred_asin, k=k), axis=1)
    average_recall_at_k = test_merged["recall@k"].mean()
    average_novelty_at_k = test_merged["novelty@k"].mean()
    
    print(f"The TI-MF has an average recall@{k}: {average_recall_at_k:.5f}, average novelty@{k}: {average_novelty_at_k:.5f}")
    
    return test_merged

# Generate N-Recommendations = {10, 25, 30, 45}

## Load Test Data

In [14]:
test = pd.read_csv(f"{DATA_PATH}/{CATEGORY}_test.csv")

In [15]:
test.head().append(test.tail())

,index,asin,title,categories,reviewerID,overall,reviewText,reviewTime,processedReviewText
0,1,1223000893,"Cat Sitter DVD Trilogy - Vol 1, Vol 2 and Vol 3",[],A39QHP5WLON5HV,5.0,There are usually one or more of my cats watch...,2013-09-14,usually cat watch tv stay trouble dvd play lik...
1,104,B00005MF9V,LitterMaid Universal Cat Privacy Tent (LMT100),"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A366V0GCEPH5CX,5.0,My cats love it and so do I. I no longer have ...,2013-02-02,cat love longer cat litter fly floor litter fl...
2,133,B00005MF9T,LitterMaid LM500 Automated Litter Box,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",ALWWS8QBYN80B,1.0,I have one female cat that weighs under 10 pou...,2004-11-17,female cat weigh pound year old use everclean ...
3,153,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A3PVI3NE7OY1SP,5.0,I love these. They make the clean up so much e...,2013-09-26,love clean easy clean box manually use issue w...
4,154,B00005MF9W,LitterMaid Waste Receptacles Automatic Litter ...,"['Pet Supplies', 'Cats', 'Litter & Housebreaki...",A2H83XMHUVDLJY,4.0,"I love this litter box. I do not use the lids,...",2014-06-26,love litter box use lid use receptacle tear cr...
41564,111601,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],AV34KNYW82YSS,4.0,Pulled lots of hair out of my Labs coat. Didn'...,2014-07-18,pulled lot hair labs coat think prove wrong co...
41565,111603,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1YMNTFLNDYQ1F,5.0,I have been trying to find a rubber bristle br...,2014-07-16,try rubber bristle brush persian year lose glo...
41566,111604,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A1FQ3HRVXA4A5B,5.0,Great product to use on your pets knowing this...,2014-07-11,great product use pet know gentle rubber damag...
41567,111605,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A3OP6CI0XCRQXO,5.0,I bought a second one because I have two cats ...,2014-07-22,buy second cat american short hair buy brush m...
41568,111606,B00KJGFGFO,Curry Brush with Coarse or Fine Bristles. High...,[],A11LC938XF35XN,5.0,Our dogs love getting brushed with this. It m...,2014-07-17,dog love brush massage remove heavy undercoat ...


In [16]:
# generating test history
test_user_history = (pd.DataFrame(test.groupby(['reviewerID'])['asin']
                                  .apply(list).reset_index()))

In [17]:
print(test_user_history)

                  reviewerID                                  asin
0      A04173782GDZSQ91AJ7OD              [B0090Z9AYS, B00CPDWT2M]
1      A042274212BJJVOBS4Q85              [B005AZ4M3Q, B00771WQIY]
2       A0436342QLT4257JODYJ  [B0018CDR68, B003SJTM8Q, B00474A3DY]
3      A04795073FIBKY8GSLZYI              [B001PKT30M, B005DGI2RY]
4      A06658082A27F4VB5UG8E              [B000TZ1TTM, B0019VUHH0]
...                      ...                                   ...
18993          AZYJE40XW6MFG              [B00HVAKJZS, B00IDZT294]
18994          AZZ56WF4X19G2                          [B004A7X218]
18995          AZZNK89PXD006  [B0002DHV16, B005BP8MQ8, B009RTX4SU]
18996          AZZV9PDNMCOZW              [B007EQL390, B00ISBWVT6]
18997          AZZYW4YOE1B6E  [B0002AQPA2, B0002AQPA2, B0002ARQV4]

[18998 rows x 2 columns]


# Preparing Dataset for Surprise's Algorithm

In [18]:
# create reader
reader = Reader(rating_scale=(1,5))
# generate data required for surprise
data = Dataset.load_from_df(train[["reviewerID", "asin", "overall"]], reader)
# generating trainset
trainset = data.build_full_trainset()

# Instantiate Pre-Initialised Matrix Factorization (Topic Modelling)

In [19]:
# instantiating ti_mf
ti_mf = cf.PreInitialisedMF(user_map=user_idx_map,
                            item_map=item_idx_map,
                            user_factor=user_factors,
                            item_factor=item_factors,
                            learning_rate=LR_ALL,
                            beta=BETA,
                            num_epochs=N_EPOCHS,
                            num_factors=50)

In [20]:
%%time
# fitting to training data
ti_mf.fit(trainset, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
CPU times: user 5min 57s, sys: 2.43 s, total: 5min 59s
Wall time: 6min 3s


In [21]:
%%time
# generate candidate items for user to predict rating
testset = trainset.build_anti_testset()

CPU times: user 46.1 s, sys: 2.18 s, total: 48.2 s
Wall time: 48.7 s


In [22]:
%%time
# predict ratings for all pairs (u, i) that are NOT in the training set
candidate_items = ti_mf.test(testset, verbose=False)

CPU times: user 12min 25s, sys: 5min 3s, total: 17min 29s
Wall time: 19min 41s


## Loop through N = {10, 25, 30, 45}

In [23]:
# generate item popularity
item_popularity = generate_item_popularity(train)

In [24]:
n_recommendations = {}
for n in [10, 25, 30, 45]:
    # retrieve the top-n items based on similarities
    top_ns = get_top_n(candidate_items, n)
    # evaluate how well the recommended items predicted the future purchases
    n_recommended_items = evaluate_recommendations(top_ns, test_user_history, item_popularity, n)
    # saving the n-value and recommended items
    n_recommendations[n] = (top_ns, n_recommended_items)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19058/19058 [01:25<00:00, 222.03it/s]


The TI-MF has an average recall@10: 0.00605, average novelty@10: 0.93035


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19058/19058 [00:59<00:00, 318.09it/s]


The TI-MF has an average recall@25: 0.01181, average novelty@25: 0.93601


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19058/19058 [01:01<00:00, 308.12it/s]


The TI-MF has an average recall@30: 0.01359, average novelty@30: 0.93856


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19058/19058 [01:05<00:00, 292.91it/s]


The TI-MF has an average recall@45: 0.01831, average novelty@45: 0.94257


# Evaluate N-Recommendations

In [25]:
def retrieve_recommendations(train: pd.DataFrame, top_ns: dict):
    """
    """
    # generating a random user
    random_user = np.random.choice(list(train['reviewerID'].unique()), 1)[0]
    print(f"For user: {random_user}:")
    print(f"Purchase History:\n{train[train['reviewerID'] == random_user][['asin', 'title']]}")

    # find the recommendations
    print(f"\nRecommending:\n")
    recommendations = (train[train['asin']
                             .isin([i[0] for i in top_ns[random_user]])][['asin', 'title']]
                       .drop_duplicates(subset='asin')
                       .set_index('asin'))
    print(f"{recommendations.loc[[i[0] for i in top_ns[random_user]]].reset_index()}")

## N=10

In [26]:
top_ns_10 = n_recommendations[10][0]
retrieve_recommendations(train, top_ns_10)

For user: A28NXWZ64PIZPJ:
Purchase History:
             asin                               title
22674  B0002I0RU8  PetSafe Busy Buddy Chuckle Dog Toy
22688  B0002I0RU8  PetSafe Busy Buddy Chuckle Dog Toy

Recommending:

         asin                                              title
0  B001LNUKE6                Purebites Cheddar Cheese Dog Treats
1  B0002DJVQY   JW Pet Company Activitoys Triple Mirror Bird Toy
2  B0021X17XS  Zymox Otic Enzymatic Pet Ear Treatment without...
3  B003BYQ100             Armarkat B5701 57-Inch Cat Tree, Ivory
4  B000F4AVPA                                Chuckit! Ultra Ball
5  B0017JFNNC                     Redbarn Naturals Bully Springs
6  B00E8GJU7Q           2 Pack Feliway Electric Diffuser (48 Ml)
7  B0012KB4D4  Purina Friskies Gravy Sensations Wet Cat Food ...
8  B000QSON4K  Greenies Pill Pockets Soft Dog Treats, Beef, C...
9  B003JFRQQ4  Scaredy Cut Tiny Trim by Small Pet Grooming Sa...


## N=25

In [27]:
top_ns_25 = n_recommendations[25][0]
retrieve_recommendations(train, top_ns_25)

For user: A2VUBIY7MXOWFD:
Purchase History:
             asin                                              title
24156  B0002RJMB4                 Safari Soft Bristle Brush for Cats
26966  B00063KGEG                   Petrodex Cat Dental Treats, 50Ct
34906  B000FPKZPA                   8in1 Perfect Coat Bath Wipes Tub
51260  B001P3NU4E  Virbac C.E.T. Enzymatic Oral Hygiene Chews for...

Recommending:

          asin                                              title
0   B003BYQ100             Armarkat B5701 57-Inch Cat Tree, Ivory
1   B0002DJVQY   JW Pet Company Activitoys Triple Mirror Bird Toy
2   B000K9JRH8  GoCat DaBird Feather Refill, Assorted Colors, ...
3   B0017JFNNC                     Redbarn Naturals Bully Springs
4   B001LNUKE6                Purebites Cheddar Cheese Dog Treats
5   B0002AS1CC                 Bergan Turbo Scratcher Accessories
6   B000F4AVPA                                Chuckit! Ultra Ball
7   B0002563S6          Magic Coat Cat Tearless Shampoo, 12-Ounce
8

## N=30

In [28]:
top_ns_30 = n_recommendations[30][0]
retrieve_recommendations(train, top_ns_30)

For user: A1SKJ0I2P6XF10:
Purchase History:
             asin                                              title
56132  B00336ET5U  Richell Paw Trax Super-Absorbent Training Pads...

Recommending:

          asin                                              title
0   B000MLHDS4  Wellness Pure Rewards Natural Grain Free Dog T...
1   B0002DJVQY   JW Pet Company Activitoys Triple Mirror Bird Toy
2   B000F4AVPA                                Chuckit! Ultra Ball
3   B001LNUKE6                Purebites Cheddar Cheese Dog Treats
4   B001HN5Z4K  Bit-O-Luv Bistro Beef Recipe Dog Treats, 4.0-O...
5   B000MLG4K2  Wellness Wellbites Soft Natural Dog Treats, Tu...
6   B0012KB4D4  Purina Friskies Gravy Sensations Wet Cat Food ...
7   B000I82DU4                 Milk-Bone Flavor Snacks Dog Treats
8   B0017JFNNC                     Redbarn Naturals Bully Springs
9   B003BYQ100             Armarkat B5701 57-Inch Cat Tree, Ivory
10  B000ILEIUE  Blue Dog Bakery | Dog Treats | All-Natural | P...
11  B000QS

## N=45

In [29]:
top_ns_45 = n_recommendations[45][0]
retrieve_recommendations(train, top_ns_45)

For user: A129E5ETY3QNBL:
Purchase History:
             asin                                              title
51966  B001TY5D6C               Cat Mate Pet Fountain - 70 Fluid Oz.
62540  B0053WMOKY  Chasing Our Tails Elk Rack Snack, 100-Percent ...
64016  B005NK5DEU                Flexi Explore Retractable Dog Leash

Recommending:

          asin                                              title
0   B000F4AVPA                                Chuckit! Ultra Ball
1   B001LNUKE6                Purebites Cheddar Cheese Dog Treats
2   B0002DJVQY   JW Pet Company Activitoys Triple Mirror Bird Toy
3   B0017JFNNC                     Redbarn Naturals Bully Springs
4   B003BYQ100             Armarkat B5701 57-Inch Cat Tree, Ivory
5   B0017J8NDY  Mammoth Flossy Chews Cottonblend Color 5-Knot ...
6   B0012KB4D4  Purina Friskies Gravy Sensations Wet Cat Food ...
7   B000I82DU4                 Milk-Bone Flavor Snacks Dog Treats
8   B003JFRQQ4  Scaredy Cut Tiny Trim by Small Pet Grooming Sa...
9   